# Notebook for cross-testing

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.io import read_image,ImageReadMode
from torch.utils.data import Dataset
import torchvision.transforms.v2 as transforms
import sys
from PIL import Image
import os

import torch.nn as nn
import torch.optim as optim
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.transforms import v2 as transforms


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


In [ ]:
cdir = !pwd
rc = 'local' if 'dg' in cdir[0] else 'colab'
if rc == 'colab':
    !echo 'COL'
else:
    !echo 'Loc'


In [ ]:
# Colab dir setup and extract
untar = 'cub_bbs'
if rc == 'colab':
    from google.colab import drive
    drive.mount('/content/gdrive/')
    if untar == 'cub':
      !tar -xvf '/content/gdrive/MyDrive/Colab Notebooks/CUB_200_2011.tgz' -C /content | tail
    if untar == 'cub_sm':
      !tar -xvf '/content/gdrive/MyDrive/Colab Notebooks/CUB_sm.tar' -C /content | tail
    if untar == 'cub_bbs':
      !tar -xzvf '/content/gdrive/MyDrive/Colab Notebooks/CUB_200_2011_bbs.tgz' -C /content | tail
    if untar == 'nab_sm':
      !tar -xzvf '/content/gdrive/MyDrive/Colab Notebooks/nabirds_sm.tgz' -C /content | tail
    if untar == 'nab_bbs':
      !tar -xzvf '/content/gdrive/MyDrive/Colab Notebooks/nabirds_bbs.tgz' -C /content | tail
      !tar -xvf '/content/gdrive/MyDrive/Colab Notebooks/CUBtxt.tar' -C /content | tail
    sys.path.append('/content/gdrive/MyDrive')



In [ ]:
from pathlib import Path
if rc == 'colab':
    datadir = Path('/content')
    untar = 'CUB'
    if untar == 'nab':
      CUBDir = Path('/content/CUB_200_2011/CUB_200_2011/')
    if untar == 'smcub':
      CUBDir = Path('/content/CUB_200_2011/')
    expdir = Path('/content/gdrive/MyDrive/birddb/expts')
    sys.path.append('/content/gdrive/MyDrive/birddb/src')
else:
    datadir = Path('/data1/datasets/birds')
    expdir = Path('/home/dg/proj/birddb/expts')
    sys.path.append('/home/dg/proj/birddb/src')


In [ ]:
import birddb
import trainer
import models
import expt

In [ ]:

dbname = 'CUB_200_2011_bbs'
myexpdir = str(expdir / ('RN50_V1/T2/' + dbname))
tdbdir = Path(datadir / dbname)
trainds = birddb.TrainImageDataset(tdbdir)

In [ ]:
import importlib
importlib.reload(expt)
from torch.utils.tensorboard import SummaryWriter
#writer = SummaryWriter(expdir / 'stats')
myexpt = expt.Expt_RN50_V1_T1(str(myexpdir),datadir,dbname,device)
model = myexpt.loadModel()
# note - makeDataloader always adds resize transform

In [ ]:
import pickle
importlib.reload(birddb)
stats = {}
statfile = myexpdir + '/stats.pkl'
print(f'statfile = {statfile}')
if os.path.exists(statfile):
    with open(statfile,'rb') as f:
        stats = pickle.load(f)
print(stats)
for db in ['nabirds_bbs']:
    if db not in stats:
        stats[db] = {}
    for test in ['train', 'test']:
        if test not in stats[db]:
            print(f'{test} not in stats[{db}]...computing')
            stats[db][test] = {}
            dbdir = Path(datadir / db)
            if db[0] == dbname[0]: # quicl test
                testds = birddb.TestImageDataset(dbdir,trainds,trainset=test)
            else:
                testds = birddb.CrossImageDataset(dbdir,trainds,trainset=test)
            testds.printStats()
            print(f'db: {db}, test: {test}')
            #myexpt.visualizeOnDataset(testds)
            loss, acc = myexpt.testOnDataset(testds)
            stats[db][test]= acc.item()
            with open(myexpdir + '/stats.pkl','wb') as f:
                pickle.dump(stats,f)
    #    stats[db][test]['loss'] = loss

      #  writer.add_scalar(dbname + '/' + db + '/' + test,acc)




In [ ]:
with open(myexpdir + '/stats.pkl','rb') as f:
    stats = pickle.load(f)
stats

In [ ]:

import pandas as pd
df = pd.DataFrame(stats)
df



In [ ]:
#import importlib
#importlib.reload(expt)
#myexpt = expt.Expt_RN50_V1_T1(expdir,datadir,dbname,device)
import glob
rows = []
for fname in glob.glob(str(expdir / '*/T2/*/*.pkl')):
    dbname = fname.split('/')[-2]
    print(dbname)
    with open(fname,'rb') as f:
        stats = pickle.load(f)
        print(stats)
        for x in stats:
            if 'wood' in x:
                continue
            row = [dbname,x,stats[x]['train'],stats[x]['test']]
            rows.append(row)


rows
df = pd.DataFrame(rows,columns=['traindb','testdb','trainacc','testacc'])
df.to_csv(expdir / 'RN50_V1/T2/stats.csv')
df

In [ ]:
myexpt.visualizeOnDataset(testds)


In [ ]:
myexpt.testOnDataset(testds)
